In [18]:
import  matplotlib
import matplotlib.pyplot as plt
import pystan

In [19]:
import numpy as np

In [20]:
import scipy.io as sio

In [21]:
df = sio.loadmat('snelson1d.mat')

In [22]:
X = df['X']
Y = df['Y']
X_t = df['tX']
Y_t = df['tY']

In [23]:
def kmm_init(X, m = 20):
    """
    This is the same initialization algorithm that is used
    in Kmeans++. It's quite simple and very useful to initialize
    the locations of the inducing points in sparse GPs.
    
    http://ilpubs.stanford.edu:8090/778/1/2006-13.pdf
    1. Take one center c1, initially chosen at random
    2. Take a new center ci with prob. \frac{D(x)^2}{\sum D(x)^2}
    3. Repeat step 2 until we have K centers.
    4. Proceed with standard K-means clustering..
    
    where D(x) is the distance to the closest cluster center. 
    If we dont want to invest time on that, just use the initialisation which is good enough atleast for the 1D case.
    
    :param X: data
    :param m: number of inducing points
    """

    # compute the distances
    XXT = np.dot(X, X.T)
    D = (-2.*XXT + np.diag(XXT)[:,np.newaxis] + np.diag(XXT)[np.newaxis,:])

    # select the first point
    s = np.random.permutation(X.shape[0])[0]
    inducing = [s]
    prob = D[s]/D[s].sum()

    for z in range(m-1):
        s = np.random.multinomial(1, prob.flatten()).argmax()
        inducing.append(s)
        prob = D[s]/D[s].sum()

    inducing = np.array(inducing)
    return X[inducing]

In [24]:
X_u = kmm_init(X)

In [25]:
print(X_u.shape)

(20, 1)


In [26]:
X_u_noisy = X_u + 0.1*np.random.rand(*X_u.shape) 

In [27]:
# X_u_noisy= X_u_noisy[X_u_noisy < 5.8] 
# X_u_noisy= X_u_noisy[X_u_noisy > 0.1] 

In [28]:
print(X.shape)

(200, 1)


In [29]:
# print(X.max()), print(X.min())
# print(X_u_noisy.max()), print(X_u_noisy.min())

In [55]:
stan_code="""
data {
int<lower=1> N;
int<lower=1> M;
real x[N];
vector[N] y;
real xu[M];
}

transformed data {
real delta = 1e-9;
}
parameters {
real<lower=0> rho;
real<lower=0> alpha;
real<lower=0> sigma;
vector[M] eta2;
}
model {
vector[N] f;
vector[N] f_mean;
matrix[N, N] f_cov;
vector[M] u;
matrix[M, M] Kuu_inv ;
{
matrix[N, N] L_K;
matrix[M, M] L_U;
matrix[N, N] K = cov_exp_quad(x, alpha, rho);

matrix[M, M] Kuu = cov_exp_quad(xu, alpha, rho);
matrix[N, M] Kfu = cov_exp_quad(x, xu, alpha, rho);

for (m in 1:M){
    Kuu[m, m] = K[m, m] + delta;
}

Kuu_inv = inverse(Kuu); 
// diagonal elements
for (n in 1:N)
K[n, n] = K[n, n] + delta;


L_K = cholesky_decompose(K);
L_U = cholesky_decompose(Kuu);
u = L_U * eta2;
# f = L_K * eta1;

f_mean = Kfu*(Kuu_inv*u);
f_cov = K - Kfu*(Kuu_inv*Kfu');

}

rho ~ inv_gamma(5, 5);
alpha ~ normal(0, 1);
sigma ~ normal(0, 1);

eta2 ~ normal(0, 1);
f ~ multi_normal(f_mean, f_cov);

y ~ normal(f, sigma);
}


"""

This code does the same thing as above, but using Cholesky decomposition.

In [56]:
stan_code2 = """
functions{
    matrix cov_cond_fn(vector u, real[] z, real[] x, real sigvar, real lengthscale, real sigma, real jitter){
        int M = size(z);
        int N = size(x);
        vector[N] f2;
        matrix[N, N] cov_f2;
        {
            matrix[N, N] L_K;
            matrix[M, M] Lu;
            vector[M] K_div_u;
            matrix[N, M] Kfu;
            matrix[N, M] v_pred;

            matrix[M, M] Kuu;
            matrix[M, M] Kuu_inv;
            vector[M] Kuu_inv_u;
            matrix[N, N] Kff;
            matrix[N, N] Qff;
            matrix[M, N] mat1;
            
            matrix[N, N] diag_delta;
            
            vector[N] f_u_mu;
            
            real sv = 1.2;
            real scale = 0.9;
            Kff = cov_exp_quad(x, sigvar, lengthscale);
            
        #    K = cov_exp_quad_ARD(x1, alpha, rho);
        
            for (i in 1:N){
                Kff[i,i] = Kff[i,i] + square(sigma);            
            }
            
            Kuu = cov_exp_quad(z, sv, scale);
            
            for (i in 1:M){
                Kuu[i,i] = Kuu[i,i] + jitter;            
            }
            
            L_K = cholesky_decompose(Kff);
            #Lu = cholesky_decompose(Kuu);

            # solving a triangular system here: Ax = b where A is a symmetric positive definite matrix.
            # LL'x = y1
            # Lc = y1
            # c = y1/L
            # c = L'x

            #K_div_u = mdivide_left_tri_low(Lu, u);
            #K_div_u = mdivide_right_tri_low(K_div_u', Lu)';
            
            Kuu_inv = inverse(Kuu);
            Kuu_inv_u = Kuu_inv*u;
            Kfu = cov_exp_quad(x, z, sigvar, lengthscale);
        #    f_u_mu = (Kfu * K_div_u);
            f_u_mu = (Kfu * Kuu_inv_u);
            #v_pred = mdivide_left_tri_low(Lu, Kfu');
            
            mat1 = Kuu_inv * Kfu';
            Qff = Kfu* mat1;
         
         #   Qff = v_pred'*v_pred;
         
            cov_f2 = Kff - Qff;
            diag_delta = diag_matrix(rep_vector(jitter, N));  
            
            # sample f from multivariate normal with mean=f2mu and covariance=cov_f2
           # f2 = multi_normal_rng(f_u_mu, cov_f2 + diag_delta);
        }
        return cov_f2;
    }
    
       vector mean_cond_fn(vector u, real[] z, real[] x, real sigvar, real lengthscale, real sigma, real jitter){
        int M = size(z);
        int N = size(x);
        vector[N] f2;
        matrix[N, N] cov_f2;
        vector[N] f_u_mu;
        {
            matrix[N, N] L_K;
            matrix[M, M] Lu;
            vector[M] K_div_u;
            matrix[N, M] Kfu;
            matrix[N, M] v_pred;

            matrix[M, M] Kuu;
            matrix[N, N] Kff;
            matrix[N, N] Qff;
            
            matrix[N, N] diag_delta;
            matrix[M, N] mat1;
            
            
            Kff = cov_exp_quad(x, sigvar, lengthscale);
            
        #    K = cov_exp_quad_ARD(x1, alpha, rho);
        
            for (i in 1:N){
                Kff[i,i] = Kff[i,i] + square(sigma);            
            }
            
            
            Kuu = cov_exp_quad(z, sigvar, lengthscale);
            for (i in 1:M){
                Kuu[i,i] = Kuu[i,i] + jitter;            
            }
            Lu = cholesky_decompose(Kuu);

            # solving a triangular system here: Ax = b where A is a symmetric positive definite matrix.
            # LL'x = y1
            # Lc = y1
            # c = y1/L
            # c = L'x

            K_div_u = mdivide_left_tri_low(Lu, u);
            K_div_u = mdivide_right_tri_low(K_div_u', Lu)';
            Kfu = cov_exp_quad(x, z, sigvar, lengthscale);
      #      K_x1_x2 = cov_exp_quad_x1_x2_ARD(x1, x2, alpha, rho);
            f_u_mu = (Kfu * K_div_u);
            #v_pred = mdivide_left_tri_low(Lu, Kfu');
            mat1 = inverse(Kuu)*Kfu';
            Qff = Kfu*mat1;
            #Qff = v_pred*v_pred';
            cov_f2 = Kff - Qff;
            diag_delta = diag_matrix(rep_vector(jitter, N));  
            
            # sample f from multivariate normal with mean=f2mu and covariance=cov_f2
#             f2 = multi_normal_rng(f_u_mu, cov_f2 + diag_delta);
        }
        return f_u_mu;
    }
}

data{
    int<lower=1> N1;
    real x1[N1];
    vector[N1] y1;
    int<lower=1> N2;
    int<lower=1> M;
    real x2[N2];
    real z[M];
}

transformed data{
    int<lower=1>N = N1+N2;
    real x[N];
    vector[N1] f1_mu = rep_vector(0, N1);
    vector[M] u_mu= rep_vector(0, M);
    real jitter = 1e-7;
    real lengthscale = 0.9;
    real sigvar = 1.5;
}

parameters{
#     real<lower=0> lengthscale;
#     real<lower=0> sigvar;
    real sigma;
    vector[M] eta;
}

transformed parameters{
#     real<lower=0> rho1 = rho[0];

    vector[N1] f1;
    vector[M] u;
    vector[M] umat;
    matrix[M, M] Lu;
    matrix[M, M] Kuu;
    matrix[M, M] Kuu_inv;
    matrix[N1, N1] cov_cond;
    vector[N1] mean_cond;
    matrix[N1, N1] Kff;
    matrix[N1,M] Kfu;

    {
        real utemp[M];
        real ftemp[N1];
        Kuu= cov_exp_quad(z, sigvar, lengthscale);
        Kfu= cov_exp_quad(x1, z, sigvar, lengthscale);
        
        for (i in 1:M){
            Kuu[i,i] = Kuu[i,i] + jitter;
        }
        for (i in 1:M){
            utemp[i] = u[i];
        }
        

     #   Lu = cholesky_decompose(Kuu);
        
        u = u_mu + Lu * eta;
        Kuu_inv = inverse(Kuu);
        umat = Kuu_inv*u;
        mean_cond = Kfu*umat;
        cov_cond = Kfu*(Kuu_inv*Kfu');
        
     #   cov_cond = cov_cond_fn(u, z, x1, sigvar, lengthscale, sigma, jitter);
     #   mean_cond = mean_cond_fn(u, z, x1, sigvar, lengthscale, sigma, jitter);
        
#         f1 = cov_cond_rng(mean_cond, cov_cond);

    }
}

model{
    lengthscale ~ inv_gamma(5,5);
    sigvar ~ normal(0,1);
    sigma ~ normal(0,1);
    eta ~ normal(0, 1);
    u ~ multi_normal(u_mu, Kuu);
    f1 ~ multi_normal(mean_cond, cov_cond);
#     f1 = f1_u*u;
    y1 ~ normal(f1, sigma);

}

generated quantities {
#   vector[N1] f1;
  vector[N2] f2;
  vector[N2] y2;

  f2 = multi_normal_rng(mean_cond, cov_cond);
#   f2 = gp_pred_rng(x2, y1, x1, alpha, rho, sigma, jitter);
  for (n2 in 1:N2){
    y2[n2] = normal_rng(f2[n2], sigma);  
  }

}

"""


In [57]:
stan_model = pystan.StanModel(model_code=stan_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_5bd300a9bfe5c0aaa6d6902a720a1a54 NOW.


In [58]:
N1 = 200

In [59]:
M = 20

In [60]:
print(np.max(X)), print(np.min(X))

5.9657729
0.059167804


(None, None)

In [61]:
N2 = 20

In [62]:
Xtest = np.linspace(0,6, N2)

In [63]:
Xtest = Xtest.reshape(-1,1)

In [64]:
Y = Y.flatten()
X = X.flatten()
Xtest = Xtest.flatten()

In [65]:
X_u_noisy = X_u_noisy.flatten()

In [66]:
lengthscale = 1.1

In [67]:
sigvar = 2.5

In [68]:
gp_sparse_regression_data = {'y1':Y, 'x1':X, 'N1':200, 'N2':N2, 'x2':Xtest, 'z':X_u_noisy, 'M':M}

In [69]:
gp_full_data = {'N':200, 'x':X, 'y':Y}
gp_full_data2 = {'N':200, 'x':X, 'y':Y, 'M':20, 'xu':X_u_noisy}

In [70]:
gp_sparse_regression_data_fixed_hyperparams = {'y1':Y, 'x1':X, 'N1':200, 'N2':N2, 'x2':Xtest, 'z':X_u_noisy, 'M':19, 'lengthscale':lengthscale, 'sigvar':sigvar }

In [81]:
# fit = stan_model.sampling(data=gp_sparse_regression_data, iter=200, chains=2)

In [82]:
fit2 = stan_model.sampling(data=gp_full_data, iter=200, chains=2)

/u/26/dhakaa1/unix/anaconda2/envs/stanpy35/lib/python3.5/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [72]:
fit3 = stan_model.sampling(data=gp_full_data2, iter=1000, chains=2)

/u/26/dhakaa1/unix/anaconda2/envs/stanpy35/lib/python3.5/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


RuntimeError: Initialization failed.